In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from batcher import Container
import cifar10
%matplotlib inline

In [2]:
ims, labels = cifar10.get_train_data()
images = Container(ims, labels)

In [3]:
test_x, test_y = cifar10.get_test_data()

In [4]:
sess = tf.InteractiveSession()

In [5]:
channels = 3

In [6]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [7]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_3x3(x):
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')

In [8]:
x = tf.placeholder(tf.float32, shape=[None, 32, 32, 3])
x_p = tf.placeholder(tf.float32)
x_drop = tf.nn.dropout(x, x_p)

y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [11]:
conv1_filters = 96

W_conv1 = weight_variable([5, 5, channels, conv1_filters])
b_conv1 = bias_variable([conv1_filters])

h_conv1 = tf.nn.relu(conv2d(x_drop, W_conv1) + b_conv1)
h_pool1 = max_pool_3x3(h_conv1)

pool1_p = tf.placeholder(tf.float32)
h_pool1_drop = tf.nn.dropout(h_pool1, pool1_p)

In [12]:
conv2_filters = 128

W_conv2 = weight_variable([5, 5, conv1_filters, conv2_filters])
b_conv2 = bias_variable([conv2_filters])

h_conv2 = tf.nn.relu(conv2d(h_pool1_drop, W_conv2) + b_conv2)
h_pool2 = max_pool_3x3(h_conv2)

pool2_p = tf.placeholder(tf.float32)
h_pool2_drop = tf.nn.dropout(h_pool2, pool2_p)

In [13]:
conv3_filters = 256

W_conv3 = weight_variable([5, 5, conv2_filters, conv3_filters])
b_conv3 = bias_variable([conv3_filters])

h_conv3 = tf.nn.relu(conv2d(h_pool2_drop, W_conv3) + b_conv3)
h_pool3 = max_pool_3x3(h_conv3)

pool3_p = tf.placeholder(tf.float32)
h_pool3_drop = tf.nn.dropout(h_pool3, pool3_p)

In [14]:
fc_units = 2048

In [15]:
W_fc1 = weight_variable([4 * 4 * conv3_filters, fc_units])
b_fc1 = bias_variable([fc_units])

h_pool3_flat = tf.reshape(h_pool3_drop, [-1, 4 * 4 * conv3_filters])
h_fc1 = tf.nn.relu(tf.matmul(h_pool3_flat, W_fc1) + b_fc1)

fc1_p = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, fc1_p)

In [16]:
W_fc2 = weight_variable([fc_units, fc_units])
b_fc2 = bias_variable([fc_units])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

fc2_p = tf.placeholder(tf.float32)
h_fc2_drop = tf.nn.dropout(h_fc2, fc2_p)

In [17]:
W_logit = weight_variable([fc_units, 10])
b_logit = bias_variable([10])

y_conv = tf.matmul(h_fc2_drop, W_logit) + b_logit

In [46]:
beta = 0.01

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv) 
#     + beta * tf.nn.l2_loss(W_fc1) 
#     + beta * tf.nn.l2_loss(W_fc2) 
#     + beta * tf.nn.l2_loss(W_logit)
)

train_step = tf.train.AdamOptimizer(1e-5).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

accuracies = []

for i in range(20000):
    batch_x, batch_y = images.next_batch(500)
    if i%100 == 0:
        train_accuracy = accuracy.eval(
            feed_dict={
                x: batch_x, 
                y_: batch_y,
                x_p: 1.,
                pool1_p: 1.,
                pool2_p: 1.,
                pool3_p: 1.,
                fc1_p: 1.,
                fc2_p: 1.
            }
        )
        print("step %d, training accuracy %g"%(i, train_accuracy))
        if i%200 == 0:
            accuracies.append(train_accuracy)
    
    train_step.run(
        feed_dict={
            x: batch_x, 
            y_: batch_y,
            x_p: 1.,
            pool1_p: 1.,
            pool2_p: 1.,
            pool3_p: 1.,
            fc1_p: .9,
            fc2_p: .9
        }
    )

step 0, training accuracy 0.106
step 100, training accuracy 0.23
step 200, training accuracy 0.322
step 300, training accuracy 0.34
step 400, training accuracy 0.372
step 500, training accuracy 0.384
step 600, training accuracy 0.356
step 700, training accuracy 0.386
step 800, training accuracy 0.412
step 900, training accuracy 0.428
step 1000, training accuracy 0.458
step 1100, training accuracy 0.46
step 1200, training accuracy 0.46
step 1300, training accuracy 0.458
step 1400, training accuracy 0.444
step 1500, training accuracy 0.43
step 1600, training accuracy 0.44
step 1700, training accuracy 0.42
step 1800, training accuracy 0.438
step 1900, training accuracy 0.44
step 2000, training accuracy 0.442
step 2100, training accuracy 0.43
step 2200, training accuracy 0.442
step 2300, training accuracy 0.424
step 2400, training accuracy 0.432
step 2500, training accuracy 0.442
step 2600, training accuracy 0.43
step 2700, training accuracy 0.466
step 2800, training accuracy 0.43
step 290

KeyboardInterrupt: 

In [ ]:
chunk_size = 500

chunk_accuracies = [
    accuracy.eval(
        feed_dict={
            x: test_x[start:start+chunk_size], 
            y_: test_y[start:start+chunk_size],
            x_p: 1.,
            pool1_p: 1.,
            pool2_p: 1.,
            pool3_p: 1.,
            fc1_p: 1.,
            fc2_p: 1.
        }
    )
    for start in range(0,len(test_y),chunk_size)
]

acc = np.mean(chunk_accuracies)

print("test accuracy %g"%acc)

In [1]:
import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

# Data loading and preprocessing
from tflearn.datasets import cifar10
(X, Y), (X_test, Y_test) = cifar10.load_data()
X, Y = shuffle(X, Y)
Y = to_categorical(Y, 10)
Y_test = to_categorical(Y_test, 10)

# Real-time data preprocessing
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

# Real-time data augmentation
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

# Convolutional network building
network = input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=img_prep)
#                     data_augmentation=img_aug)
network = conv_2d(network, 96, 5, activation='relu')
network = max_pool_2d(network, 3)
# network = conv_2d(network, 128, 5, activation='relu')
# network = max_pool_2d(network, 3)
# network = conv_2d(network, 256, 5, activation='relu')
# network = max_pool_2d(network, 3)
network = fully_connected(network, 2048, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 10, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

# Train using classifier
model = tflearn.DNN(network, tensorboard_verbose=0)

In [2]:
model.fit(X, Y, n_epoch=50, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=96, run_id='cifar10_cnn')

Training Step: 26049  | total loss: 0.19484 | time: 17.414s
| Adam | epoch: 050 | loss: 0.19484 - acc: 0.9403 -- iter: 49920/50000
Training Step: 26050  | total loss: 0.19518 | time: 18.831s
| Adam | epoch: 050 | loss: 0.19518 - acc: 0.9400 | val_loss: 1.49725 - val_acc: 0.7172 -- iter: 50000/50000
--


In [3]:
model.fit(X, Y, n_epoch=50, shuffle=True,
          show_metric=True, batch_size=96, run_id='cifar10_cnn')

Training Step: 42209  | total loss: 0.44914 | time: 0.272s
| Adam | epoch: 082 | loss: 0.44914 - acc: 0.9358 -- iter: 00768/50000


KeyboardInterrupt: 